In [1]:
!pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.6 MB/s eta 0:00:00


In [2]:
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import AutoTokenizer,AutoModelForSequenceClassification, AutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments, Seq2SeqTrainer, Seq2SeqTrainingArguments
import numpy as np
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import os 
os.environ["WANDB_DISABLED"] = "true"

In [3]:
import re
def clean(data):
    with open('dataset/teencode.txt','r') as file:
      file = file.read()
      lines = file.split('\n')
      for line in lines:
        elements = line.split('\t')
        data = re.sub(r'\b{}\b'.format(elements[0]), elements[1], data)
    data = re.sub('per ','',data)
    data = re.sub(r'\s+', ' ', data)
    return data
  # print(s)

In [5]:
import re
# call the function
df_train = pd.read_excel('dataset/train.xlsx')
df_test =  pd.read_excel('dataset/test.xlsx')
df_valid = pd.read_excel('dataset/valid.xlsx')

df_train['Sentence'] = df_train['Sentence'].apply(clean)
df_test['Sentence'] = df_test['Sentence'].apply(clean)
df_valid['Sentence'] = df_valid['Sentence'].apply(clean)



test_texts = list(df_test['Sentence'])
train_texts = list(df_train['Sentence'])
valid_texts = list(df_valid['Sentence'])

y= LabelEncoder()

train_labels = y.fit_transform(df_train['Emotion'])
valid_labels = y.fit_transform(df_valid['Emotion'])
test_labels = y.fit_transform(df_test['Emotion'])

target_names = list(df_train.Emotion.unique())

In [6]:
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf
        tf.random.set_seed(seed)

set_seed(1)

# the model we gonna train, base uncased BERT
# check text classification models here: https://huggingface.co/models?filter=text-classification
 
model_name = "xlm-roberta-base" 





# max sequence length for each document/sentence sample
max_length = 512
# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [7]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=max_length)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length)


# https://huggingface.co/transformers/v3.4.0/custom_datasets.html
class NewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset

train_dataset = NewsGroupsDataset(train_encodings, train_labels)
valid_dataset = NewsGroupsDataset(valid_encodings, valid_labels)
test_dataset = NewsGroupsDataset(test_encodings, test_labels)
# load the model and pass to CUDA
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7).to("cuda")

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

In [8]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # calculate accuracy using sklearn's function
    acc = accuracy_score(labels, preds)
    return {
      'accuracy': acc,
    }

training_args = TrainingArguments(
    output_dir='my_model',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=15,  # batch size per device during training
    per_device_eval_batch_size=15,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    learning_rate=4e-5

)

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)
# train the model
trainer.train()
# evaluate the current model after training
trainer.evaluate()
# saving the fine tuned model & tokenizer

# model_path = "..output/vinai/phobert-base"
# model.save_pretrained(model_path)
# tokenizer.save_pretrained(model_path)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5548
  Num Epochs = 5
  Instantaneous batch size per device = 15
  Total train batch size (w. parallel, distributed & accumulation) = 15
  Gradient Accumulation steps = 1
  Total optimization steps = 1850
  Number of trainable parameters = 278049031
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad

Step,Training Loss
500,1.592200
1000,1.101900
1500,0.763100


Saving model checkpoint to my_model/checkpoint-500
Configuration saved in my_model/checkpoint-500/config.json
Model weights saved in my_model/checkpoint-500/pytorch_model.bin
tokenizer config file saved in my_model/checkpoint-500/tokenizer_config.json
Special tokens file saved in my_model/checkpoint-500/special_tokens_map.json
Saving model checkpoint to my_model/checkpoint-1000
Configuration saved in my_model/checkpoint-1000/config.json
Model weights saved in my_model/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in my_model/checkpoint-1000/tokenizer_config.json
Special tokens file saved in my_model/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to my_model/checkpoint-1500
Configuration saved in my_model/checkpoint-1500/config.json
Model weights saved in my_model/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in my_model/checkpoint-1500/tokenizer_config.json
Special tokens file saved in my_model/checkpoint-1500/special_tokens_map.json


T

{'eval_loss': 1.3747104406356812,
 'eval_accuracy': 0.5860058309037901,
 'eval_runtime': 3.6914,
 'eval_samples_per_second': 185.84,
 'eval_steps_per_second': 12.462,
 'epoch': 5.0}

In [9]:
from sklearn.metrics import classification_report
import numpy as np
pred = trainer.predict(test_dataset)

# print(test_labels)
y_pred = np.argmax(pred.predictions, axis=1)
# print(y_pred)
print(classification_report(test_labels,y_pred))

***** Running Prediction *****
  Num examples = 693
  Batch size = 15


              precision    recall  f1-score   support

           0       0.47      0.53      0.49        40
           1       0.62      0.60      0.61       132
           2       0.76      0.73      0.75       193
           3       0.66      0.83      0.73        46
           4       0.61      0.58      0.60       129
           5       0.72      0.76      0.74       116
           6       0.62      0.57      0.59        37

    accuracy                           0.67       693
   macro avg       0.64      0.66      0.64       693
weighted avg       0.67      0.67      0.67       693



In [11]:
trainer.save_model('trainer-bert')

Saving model checkpoint to trainer-bert
Configuration saved in trainer-bert/config.json
Model weights saved in trainer-bert/pytorch_model.bin
tokenizer config file saved in trainer-bert/tokenizer_config.json
Special tokens file saved in trainer-bert/special_tokens_map.json
